Help by Loïc Signer (Sciper: 289076)

In [15]:
import numpy as np
import base64
import hashlib
import string
import itertools
import difflib

# Exercise 1: Lateralus

In [16]:
# Use of fast fibonacci formula and the fact that F_(n+k) = F_n * F_k + F_(n-1) * F_(k-1)
def KeyAgreement(pkB, skA):
    f_n, f_m, p = pkB
    k = skA
    matrix = Matrix([[mod(1, p),mod(1, p)], [mod(1, p), mod(0, p)]])^k
    f_k = matrix[0, 0]
    f_l = matrix[0, 1]
    return ((f_n*f_k) + (f_m*f_l))%p

In [17]:
pkb = [95195310954889373512345438378131575937054828655845, 
       43479811007365370107208279544354021551032671119905, 
       766215663951334104896602260678067980826147578172259]
sk = 2067965692858887702129329
res = KeyAgreement(pkb, sk)
print(res)

130835659930086979118817099811520250516735013953538


# Exercise 2 : VGhlIGJhc2Ugb2YgdHJhbnNwb3NpdGlvbg==

In [18]:
# Function used to create all possible permutations 
def rotationKey(key):
    lengthKey = len(key)
    rotKey = np.empty((lengthKey, lengthKey), dtype=int)
    for i in range(lengthKey):
        tmp = []
        tmp.extend(key[lengthKey - i:])
        tmp.extend(key[:lengthKey - i])
        
        rotKey[i] = tmp
    return rotKey

In [19]:
def decodeRotation(key1, key2, cypher):    
    #substract by one to have correct indices
    rotKey1 = rotationKey(key1) - 1
    rotKey2 = rotationKey(key2) - 1
    
    lengthCypher = len(cypher)
    lengthKey2 = len(key2)
    lengthKey1 = len(key1)
    ratio2 = lengthCypher//lengthKey2
    ratio1 = lengthCypher//lengthKey1
    
    #putting the cypher in the array 
    ct_array = np.empty(lengthKey2, dtype=('U', ratio2))
    ct_array = np.empty(lengthKey2, dtype=('U', ratio2))
    count = 0
    for i in range(ratio2, lengthCypher+ratio2, ratio2):
        start = i - ratio2
        end = i
        cut = cypher[start:end]
        ct_array[count] = cut
        count += 1
            

    #creating all rotations of the cypher array.
    allCt = []
    for rot in rotKey2:
        tmp = ct_array[rot]
        ct_ = ""
        for j in range(ratio2):
            for i in tmp:
                ct_ += i[j]
        allCt.append(ct_)
        
    #repeating the same process but instead of of doing it with one cypher we doing it with  all the previous computed rotations.
    ct_array = np.empty((len(allCt),lengthKey1), dtype=('U', ratio1))
    for i, ct in enumerate(allCt):
        count = 0
        for j in range(ratio1, lengthCypher+ratio1, ratio1):
            start = j - ratio1
            end = j
            cut = ct[start:end]
            ct_array[i][count] = cut
            count += 1
                
    allCt = []
    for ct in ct_array:
        for rot in rotKey1:
            tmp = ct[rot]
            ct_ = ""
            for j in range(ratio1):
                for i in tmp:
                    ct_ += i[j]
            allCt.append(ct_)

    result = [base64.b64decode(x.encode('utf-8') + b'==') for x in allCt]
    return result

In [20]:
key1 = [13, 10, 5, 2, 8, 12, 1, 14, 3, 11, 6, 7, 4, 9]
key2 = [1, 7, 6, 2, 5, 4, 3]
cypher = "WLvWDgHgB sdQ9bhcgB C F21G2lXal bxblZpwZlB lWW0CIJWZ b5IRGaBmdRXo W0dIgRclb apRZD5I1G0ui NiZJbNYRW NyiLp osmyBGl4c=IBXp9Ho4WkWH=HYkHaMdAZ 2b"
res = decodeRotation(key1, key2, cypher)

# Exercise 3: The Return of GEDEFU

In [21]:
def decryptionGEDEFU(cy, key, lenTxt):
    #initialisation of all used variables 
    C = {'A':0, 'D':1, 'F':2, 'G':3, 'V':4, 'X':5}
    sub, pad, rot = key
    lengthRot = len(rot)
    lengthCy = len(cy)
    ratio = lengthCy//lengthRot
    elems = [''] * lengthRot
    
    #creating the inverse of the rotation
    combo = dict(zip(rot, range(len(rot))))
    invRot = [combo[i] for i in range(len(rot))]
    
    #cutting the cypher to put it in the array
    count = 0
    for i in range(ratio, lengthCy+ratio, ratio):
        start = i - ratio
        end = i
        cut = cy[start:end]
        elems[count] = cut
        count += 1
    
    #inverse rotation of the array of slice of cypher
    elems = [elems[i] for i in invRot]
    tmpString = ''
    
    for j in range(ratio):
        for elem in elems:
            tmpString += elem[j]
     
    #searching for a padding or not and add to the strList all strings with a padding and unpad
    strList = []
    if len(tmpString)%2 == 0:
        strList.append(tmpString)
    
    count = -1
    while tmpString[count] == pad:
        cut = tmpString[:count]
        if len(cut)%2 == 0:
            strList.append(cut)
        count -= 1
    
    plaintexts = [''] * len(strList)
    
    #decyphering all the strings 
    for index, string in enumerate(strList):
        for i in range(2, len(string)+2, 2):
            cut = string[i-2:i]
            fst = C[cut[0]]
            scd = C[cut[1]]
            plaintexts[index] += sub[6*fst + scd]
    
    #returning the element with the same length as the given one.
    return [x for x in plaintexts if len(x) == lenTxt][0]

In [22]:
sub = 'MP8ZTQYC0E5OLAFXK3RIH749WBJ6U2SGNV1D'
pad = "G"
rot = (0, 1, 4, 5, 3, 2, 6)
key = (sub, pad, rot)
cy = "DADXFGFGXDFFXGGAFFFGAGGDFGADAXVDDFG"
lenTxt = 16
decryptionGEDEFU(cy, key, lenTxt)

'ERLWLHAEENOSEFAN'

In [23]:
def encryptGEDEFU(sk, x):
    C = ['A', 'D', 'F', 'G', 'V', 'X']
    sub, pad, rot = sk
    lengthRot = len(rot)
    substitution = ''
    
    #substitution of the plaintext 
    for char in x:
        if char not in sub:
            substitution += char * 2
        else:
            index = sub.find(char)
            fst = index//6
            scd = index % 6
            substitution += C[fst] + C[scd]
    
    add_pad = len(substitution)%lengthRot
    
    #add padding
    if add_pad != 0:
        substitution += pad * (lengthRot-add_pad) 
    
    #slice the pad substitution in an array
    padding = [''] * lengthRot
    for i in range(lengthRot):
        for j in range(0,len(substitution),lengthRot):
            padding[i] += substitution[j + i]
    
    cypher = ''
    
    #output the cypher by adding the slice the way the algorithm told us to do
    for i in rot:
        cypher += padding[i]
    
    return cypher

In [24]:
sub= 'HTBWZYJ5Q0KV96P3RLX84UONSE1GF2ACI7DM'
pad= "D"
rot= (8, 9, 1, 11, 5, 0, 12, 10, 6, 3, 7, 4, 2)
sk = (sub, pad, rot)
x = "WELCOME TO THE BONFIRE, UNKINDLED ONE. I AM A FIRE KEEPER. I TEND TO THE FLAME, AND TEND TO THEE. THE LORDS HAVE LEFT THEIR THRONES, AND MUST BE DELIVER'D TO THEM. TO THIS END, I AM AT THY SIDE."

encryptGEDEFU(sk, x)

"G G FD DFDAAX V ADXVGDFVV GAADVAXGX.  VGDXVADFAVFGX X DAXXA.G DFV FVDX  ,V AVV AAXDVV F AVXAVGD AD.XAV G.GAAVVVAFAXAV XDXGF,XGAFFAVFAXAVAFAF VV'DG   XADVFDVX V  D,D  FX  V VXG.X,X VAXXX  V VVDAV VDD D FDD.A,  DXDVGV.X .XAX D.XXVFXX X XDXXA.XVG,XXXVVD XGVVD XVVXAV'AVV AVDA VFG XD DV GDAV DA,G F AAXDXD V VVXVDV XX D  VDGAADXA DXDVFDADGVXFF GVXXAAV AA GXVAD F FV  VX  VVFA  DADXDAD,X D  V AA"

In [31]:
def decryptionMd5(sk, cy, plain_help, cy_help, md5Hash):
    #Initialisation of the alphabet A, the special characters P and the substitution characters C.
    #Replacing the '\x00' in the incomplete substitution by a character not in P and A to have a 36 long substitution matrix.
    P = string.punctuation + string.whitespace
    A = string.ascii_uppercase + string.digits
    C = ['A', 'D', 'F', 'G', 'V', 'X']
    char = 'a'
    incomplete_sub, pad = sk
    incomplete_sub = incomplete_sub.replace("\x00", char)
    sub = ''
    
    #The first part of the code is just about finding the good permutation and the complete substitution matrix.
    
    #Encrypting the plaintext helper with the incomplete substitution matrix if the element is neither in A and P we used the char not in both alphabet initialised at the beginning.
    for elem in plain_help:
        if elem in P:
            elem += elem * 2
        else:
            index = incomplete_sub.find(elem)
            if index != -1:
                fst = index//6
                scd = index%6
                sub += C[fst] + C[scd]            
            else:
                sub += char * 2
    
    #if the incomplete cypher we have has a smaller length than its original cypher we add a padding
    if len(sub) < len(cy_help):
        sub += pad * (len(cy_help) - len(sub))
    
    #Now we using the given cypher and try to reverse it to have the cypher equivalent to the one we find before
    #Create all cypher matrices with the size the permutation matrix can take.
    all_per_cy = []
    for i in range(2, 7):
        res = [''] * i
        count = 0
        ratio = len(sub)//i
        for j in range(ratio, len(cy_help) + ratio, ratio):
            start = j - ratio
            end = j
            cut = cy_help[start:end]
            res[count] = cut
            count += 1
            all_per_cy.append(res)  
        
    maxProba = 0
    finalCy = ''
    invPerm = []
    
    #With all permutations cypher, now we use the 'itertools' library to create all permutations of size permutation cypher.
    #To find the most likely cypher we use the 'difflib' library with the method SequenceMatcher to compute the Hamming distance and the string with the smallest Hamming distance is most likely to be the wanted cypher.
    for per_cy in all_per_cy:
        permutations = list(itertools.permutations(range(len(per_cy))))
        for permutation in permutations:
            decypher = ''
            for i in range(len(per_cy[0])):
                for p in permutation:
                    decypher += per_cy[p][i]
            diff = difflib.SequenceMatcher(None, sub, decypher).ratio()
            if diff > maxProba:
                maxProba = diff
                finalCy = decypher
                invPerm = permutation
    
    all_sub = list(incomplete_sub)
    
    #Once we have the good cypher we compare the incomplete cypher we have and the cypher we found previously.
    #And we complete the substitution matrix.
    for i in range(len(plain_help)):
        if plain_help[i] not in all_sub and plain_help[i] not in P:
            cut = finalCy[2*i:2*(i+1)]
            fst, scd = cut[0], cut[1]
            index = 6*C.index(fst) + C.index(scd)
            all_sub[index] = plain_help[i]
    
    missing_char = [x for x in A if x not in all_sub]
    
    all_sub_matrices = []
    
    #After completing the substitution matrix, some letters could be missing. 
    #In this case we create all possible substitution matrices with the missing letters if not the case we continue with our complete substitution matrix.
    if len(missing_char) != 0:
        permutations = list(itertools.permutations(range(len(missing_char))))
        for permutation in permutations:
            tmp_sub = all_sub.copy()
            for p in permutation:
                tmp_sub[tmp_sub.index(char)] = missing_char[p]
            all_sub_matrices.append(tmp_sub)
    else:
        all_sub_matrices.append(all_sub)
    
    # Decryption of the cypher 
    
    lengthPerm = len(invPerm)
    lengthCy = len(cy)
    res_matrix = [''] * lengthPerm
    ratio = lengthCy//lengthPerm
    count = 0
    for i in range(ratio, lengthCy + ratio, ratio):
        start = i - ratio
        end = i
        cut = cy[start:end]
        res_matrix[count] = cut
        count += 1
        
    res_matrix = [res_matrix[i] for i in invPerm]
    tmpString = ''
    for i in range(ratio):
        for char in res_matrix:
            tmpString += char[i]
    
    strList = []
    if len(tmpString)%2 == 0:
        strList.append(tmpString)
    
    count = -1
    while tmpString[count] == pad:
        cut = tmpString[:count]
        if len(cut)%2 == 0:
            strList.append(cut)
        count -= 1
    
    #We try all the substitution matrices found before and to be sure to have the good result, we comparing our finding the the given hash of the searching plaintext.
    #If the substitution matrices and the decryption part were well coded we must find a result.
    for sub_matrix in all_sub_matrices:
        for index, chars in enumerate(strList):
            plaintext = ''
            for i in range(0, len(chars), 2):
                cut = chars[i:i+2]
                fst, scd = cut[0], cut[1]
                if fst in P and scd in P:
                    plaintext += fst
                else:
                    
                    plaintext += sub_matrix[6*C.index(fst) + C.index(scd)]
            s = plaintext.encode()
            r = base64.b64encode(s)
            H = hashlib.md5(r).hexdigest()
            if H == md5Hash:
                return plaintext
    print('fail')
    

In [28]:
plain_help = "OBI-WAN: IT'S OVER, ANAKIN! I HAVE THE HIGH GROUND! ANAKIN SKYWALKER: YOU UNDERESTIMATE MY POWER! OBI-WAN: DON'T TRY IT."
cy_help = "DGFX: D' GDD XAVA!D AG DD DF FAAV!AVFG GGFAVV:GG AVXDGDFDXXVGGDD GD-AV XA' DG D.AV-FA:DDG XX, AXDV GVXD VXVGV VGAX XAVA FVAADD VF AXVXDGAD F DFX! DGFX: AVD VVDDGD-AV G'GAGV,AVFG!  FXXDF FDFDDFV! AXDVGVFXFX: AAFVDDGDXXD G AFV!AV-FA:VG'DDD G."
cy = " AAFDGD DGDVA AAFVXX, ADAFG.. FDAG FG AFAVV VXDXAADAVXFVDA DD ADDGA X VXD AADGDGAAAVAFGDV GGDF G G XGDDX FGGD A DGAG DD ADDDFGV  VAAAX G XAXG ADAXVGG,DF FDXG VXD DXVXDG FG DGGX XA VGGGD A VXGFAAGAAX G VDG. VGD VXXGDDGGAVDF DGAAG XVAGGG DVDX DDGAAVFAFXFDVAVDGAD, VXGAVXD VXXXAG FG DADDG DGADVX DA DX VDFDG DD FXVGAGAVGXXXAV VGDXXDVDF FAXG,DGGXDA DD AVAGD...A AV DXXAF G XDXAD VV,XX XAV..DAXGA AGDVFADXDF FAXDG DGAAG. VXDXADAVAVDDVXVGDX D GDADV GADGVDXG G A DGFD. ADAFDGVXAAD VXXXDX FDFA AGFXD X DXAVDXDAGFGG DD VXVVDDVXXXDDDGA AGGDGDV AAGFGDAFDGDF GAA DAVD X AVA.FFAXDF AAFDG G DD DDDAX AXDGAVVXADG VG XADDV AAG G XDVG,DF GGDXXXFAAAAV AGGXVXAVVGGXXFV FGVXD AGVD, VXDAVXAV G DAA XV FGDA G DD AAFV D GDDD VXXGXAXD..XAGD DF XA A  FAXG AD VV FX.. DVFDAVX XDAXX DD AAXDX DDDAX.DF DFXDG G DAX DFDG A FDGGD DX GGDXGDGDVVGGAD.VGDV D DGGXXDF FDDV DD XDAXVVXAVVXVA VDDDGDD, VXDDDX DAX VVXADAVVX GADDXAA GDDDV D DD XXA AXDDAVAXAG FDA DD AVVXG X VXVXFVDADVAXGA XDXDGDFG AXXGD VXA X XXDG DD FDDF AAVDAADVX GXDDA XGDDADXDF VVGFVDFG,DF FADDXG X DXD AXVADGD A VXDXAD, A DDAXDF AAVDA..."
incomplete_sub = 'L68O\x00\x009\x00\x00I\x00\x00\x00\x00\x00Z\x0023\x007S\x0004RH1\x00\x00\x00Q\x00V\x005'
pad = "V"
md5Hash = "e045b7dc708ea75fc8bcf6b0f1b4d090"
sk = (incomplete_sub, pad)

decryptionMd5(sk, cy, plain_help, cy_help, md5Hash)

'A LONG TIME AGO IN A GALAXY FAR, FAR AWAY... TURMOIL HAS ENGULFED THE GALACTIC REPUBLIC. THE TAXATION OF TRADE ROUTES TO OUTLYING STAR SYSTEMS IS IN DISPUTE. HOPING TO RESOLVE THE MATTER WITH A BLOCKADE OF DEADLY BATTLESHIPS, THE GREEDY TRADE FEDERATION HAS STOPPED ALL SHIPPING TO THE SMALL PLANET OF NABOO. WHILE THE CONGRESS OF THE REPUBLIC ENDLESSLY DEBATES THIS ALARMING CHAIN OF EVENTS, THE SUPREME CHANCELLOR HAS SECRETLY DISPATCHED TWO JEDI KNIGHTS, THE GUARDIANS OF PEACE AND JUSTICE IN THE GALAXY, TO SETTLE THE CONFLICT....'

In [29]:
#Work the same way as the previous algorithm but without the permutation part because it was given this time.
def decypherMd5(sk, cy, plain_help, cy_help, md5Hash):
    P = string.punctuation + string.whitespace
    A = string.ascii_uppercase + string.digits
    C = ['A', 'D', 'F', 'G', 'V', 'X']
    char = '?'
    incomplete_sub, pad, permutation = sk
    incomplete_sub = incomplete_sub.replace("\x00", char)
    
    combo = dict(zip(rot, range(len(rot))))
    invRot = [combo[i] for i in range(len(rot))]
    lengthInvRot = len(invRot)
    
    sub = ''
    for i in plain_help:
        if i in P:
            sub += i * 2
        else:
            index = incomplete_sub.find(i)
            if index != -1:
                fst = index//6
                scd = index%6
                sub += C[fst] + C[scd]            
            else:
                sub += char * 2
    
    if len(sub) < len(cy_help):
        sub += pad * (len(cy_help) - len(sub))
    
    
    count = 0
    ratio = len(cy_help)//len(invRot)
    res = [''] * len(invRot)
    for i in range(ratio, len(cy_help) + ratio, ratio):
        start = i - ratio
        end = i
        cut = cy_help[start:end]
        res[count] = cut
        count += 1  
    
    res = [res[i] for i in invRot]
    
    decypher = ''
    for i in range(ratio):
        for elem in res:
            decypher += elem[i]
    
    all_sub = list(incomplete_sub)
    
    for i in range(len(plain_help)):
        if plain_help[i] not in all_sub and plain_help[i] not in P:
            cut = decypher[2*i:2*(i+1)]
            fst, scd = cut[0], cut[1]
            index = 6*C.index(fst) + C.index(scd)
            all_sub[index] = plain_help[i]
    
    missing_char = [x for x in A if x not in all_sub]
    
    all_sub_matrices = []
    
    permutations = list(itertools.permutations(range(len(missing_char))))
    for permutation in permutations:
        tmp_sub = all_sub.copy()
        for p in permutation:
            tmp_sub[tmp_sub.index(char)] = missing_char[p]
        all_sub_matrices.append(tmp_sub)
    
    # Decryption of the cypher
    lengthCy = len(cy)
    res_matrix = [''] * lengthInvRot
    ratio = lengthCy//lengthInvRot
    count = 0
    for i in range(ratio, lengthCy + ratio, ratio):
        start = i - ratio
        end = i
        cut = cy[start:end]
        res_matrix[count] = cut
        count += 1
        
    res_matrix = [res_matrix[i] for i in invRot]
    tmpString = ''
    for i in range(ratio):
        for char in res_matrix:
            tmpString += char[i]
    
    strList = []
    if len(tmpString)%2 == 0:
        strList.append(tmpString)
    
    count = -1
    while tmpString[count] == pad:
        cut = tmpString[:count]
        if len(cut)%2 == 0:
            strList.append(cut)
        count -= 1
    
    for sub_matrix in all_sub_matrices:
        for index, chars in enumerate(strList):
            plaintext = ''
            for i in range(0, len(chars), 2):
                cut = chars[i:i+2]
                fst, scd = cut[0], cut[1]
                if fst in P and scd in P:
                    plaintext += fst
                else:
                    plaintext += sub_matrix[6*C.index(fst) + C.index(scd)]
            s = plaintext.encode()
            r = base64.b64encode(s)
            H = hashlib.md5(r).hexdigest()
            if H == md5Hash:
                return plaintext
    print('fail')

In [30]:
plain_help="OBI-WAN: IT'S OVER, ANAKIN! I HAVE THE HIGH GROUND! ANAKIN SKYWALKER: YOU UNDERESTIMATE MY POWER! OBI-WAN: DON'T TRY IT."
cy_help="G:  XDF GAGF FXA DXGF:  DDV GAXX VFF DDXGDF,DAAADDAVVXD !GGXD GD!A A!DG:VFVGGD' D'XA FAVDFAGVGXX-GF.-DAD VAX DD DF GF:FDGFVDD DGD VXGDGVD  FXXFDF FFD GGGFAFF  XD'FA XGAXFAFFAGF-GX.-FVX FFF XD XX XG:XFFXF,FFFFXDAFAFD !XFFF FD!V G!FX:AFFFFX' "
cy="GFXXF FFFGDFXGVGDFD GGAVDVFVXXDF VFXFDVF FVVG FD AV DAAG GV,G F VV G VDX F XGV ADDDFX GVXVDXXDFDF.F AFGFFFFFFXF. AA,F F FG G VDF G FGF VXFFXVGFF AXVXDGG AFAG VD VF FFFF"
incomplete_sub='\x00\x00\x00\x00\x00\x00J\x00\x00ZX\x00\x005\x00\x00\x00\x00\x003\x00\x002W\x004\x00\x000769\x00\x00\x00Q'
pad="V"
rot=(3, 2, 0, 4, 10, 6, 8, 9, 11, 7, 1, 5)
md5Hash="39ccbff1b943e27da35b2718cc6fe4fe"
sk = (incomplete_sub, pad, rot)


decypherMd5(sk, cy, plain_help, cy_help, md5Hash) 

'CLEAR YOUR MIND MUST BE, IF YOU ARE TO DISCOVER THE REAL VILLAINS BEHIND THIS PLOT.'